In [1]:
import json
import yaml
import psycopg2
import requests
import os

# **Connect to Database**

In [2]:
def make_db_connection_psycopg2(database: str, autocommit: bool = False):

    #assert database in list(conn_dict.keys()), "server couldn't be recognized in the config file"

    
    database="blog"
    host="localhost"
    port="5432"
    user="postgres"
    password="matin9862"
    
    conn = psycopg2.connect(user=user, password=password, host=host, dbname=database)
    cursor = conn.cursor()

    if autocommit:
        conn.autocommit = True

    return conn, cursor


In [4]:
conn, curs = make_db_connection_psycopg2(database="blog")

In [5]:
conn

<connection object at 0x7f5580ae8400; dsn: 'user=postgres password=xxx dbname=blog host=localhost', closed: 0>

In [6]:
conn, cursor = make_db_connection_psycopg2("blog")
# query = """
# SELECT * FROM information_schema.tables WHERE table_schema = 'public';
# "

query = """
SELECT * FROM home_blog;

"""

cursor.execute(query)
rows = cursor.fetchall()

for i in rows:
    print(i)

(1, 'rings', 'rings of lords', 'c.zhzkhdh.vx.dkbvxvxv', datetime.datetime(2024, 1, 11, 19, 1, 3, 266020, tzinfo=datetime.timezone.utc), datetime.time(19, 1), 1)


# **Ollama**

In [7]:
def call_llama(model, prompt, stream=False):
    url = "http://localhost:11434/api/generate"
    data = {
        "model": model,
        "prompt":prompt,
        "stream":stream
    }
    
    json_data = json.dumps(data)
    response = requests.post(url, data=json_data, headers={"Content-Type":"application/json"})
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: Status code is: {response.status_code}"

In [9]:
call_llama('llama2', "hi")['response']

"Hi there! It's nice to meet you. How are you today?"

## **Prompt**

In [41]:
prompt = ''' 
        Please suggest 2 topics in the field of machine learning for me.
        this suggestion must be this format (json):
        {"title": YOUR SUGGESTION,
        "abstract":YOUR SUGGESTION,
        "content":YOUR SUGGESTION,
        "date_posted":YOUR SUGGESTION,
        "author":YOUR SUGGESTION,
        "reading_time":YOUR SUGGESTION,
        }
 '''

## **Call llama**

In [42]:

text_idea_ai = call_llama('llama2', prompt)['response']

In [43]:
from pprint import pprint
print(text_idea_ai)

Sure! Here are two potential topics in the field of machine learning that you might find interesting:

{
"title": "Explainable AI: The Future of Machine Learning",
"abstract": "As machine learning models become increasingly complex, it is becoming more important to understand how they make decisions. Explainable AI is a rapidly growing field that aims to provide insights into the decision-making process of AI models. In this topic, we will explore the current state of explainable AI, its applications, and the challenges it faces.",
"content": "In recent years, there has been growing concern about the lack of transparency in machine learning models. Explainable AI is a response to this concern, providing a way to understand how AI systems make decisions. In this topic, we will discuss the current state of explainable AI, including its applications and challenges. We will also explore the latest techniques and tools being developed to support explainable AI.",
"date_posted": "March 2023"

In [ ]:
title
abstract
content
date_posted
author
reading_time

In [44]:
from pprint import pprint
type(json.dumps(text_idea_ai))

<class 'str'>

In [49]:
import re
import ast
# Define the regex pattern for detecting dictionaries
pattern = r'\{[^{}]*\}'

# Find all matches in the content
matches = re.findall(pattern, text_idea_ai, re.DOTALL)

# Print the extracted dictionaries
list_of_blog = [ast.literal_eval(match) for match in matches]

In [52]:
list_of_blog[1]

{'title': 'Machine Learning for Natural Language Processing', 'abstract': 'Natural language processing (NLP) is a rapidly growing field that uses machine learning to analyze and generate human language. In this topic, we will explore the current state of NLP, including its applications and challenges. We will also discuss the latest techniques and tools being developed to support NLP, including deep learning models and attention mechanisms.', 'content': 'Natural language processing (NLP) is a rapidly growing field that uses machine learning to analyze and generate human language. In this topic, we will explore the current state of NLP, including its applications and challenges. We will also discuss the latest techniques and tools being developed to support NLP, including deep learning models and attention mechanisms.', 'date_posted': 'February 2023', 'author': 'Jane Smith', 'reading_time': '90 minutes'}

In [53]:
title = list_of_blog[1].get("title")
abstract = list_of_blog[1].get("abstract")
content = list_of_blog[1].get("content")
date_posted = list_of_blog[1].get("date_posted")
author = list_of_blog[1].get("author")
reading_time = list_of_blog[1].get("reading_time")

'Natural language processing (NLP) is a rapidly growing field that uses machine learning to analyze and generate human language. In this topic, we will explore the current state of NLP, including its applications and challenges. We will also discuss the latest techniques and tools being developed to support NLP, including deep learning models and attention mechanisms.'